# <font color=blue>Mean Machine - LASSO Regression (coordinate descent)<font>

In this notebook, we will implement LASSO regression via coordinate descent:
* write a function to normalize features
* implement coordinate descent for LASSO
* explore effects of L1 penalty

# 1: Set the path that includes all relevant files

In [1]:
import os                                               # 操作系统用来管理文件路径
os.chdir("C:\Users\stevenwsy\Desktop\Steven - Python")  # 将路径改为 "C:\Users\stevenwsy\Desktop\Steven - Python"

# 注意：你们要把 "C:\Users\stevenwsy\Desktop\Steven - Python" 改成在你们电脑里面的路径

# 2: Import all relevant packages

In [2]:
import numpy as np     # 用来做数学运算
import pandas as pd    # 用来处理数据表

# 将所有图都在 Notebook 里显示
%matplotlib inline               
import matplotlib.pyplot as plt  # 用来画图

from sklearn.cross_validation import train_test_split     # 做交叉验证，划分训练集和测试集

# 3: Load in house data

In [3]:
sales = pd.read_csv('kc_house_data.csv')
sales['floors'] = sales['floors'].astype(int)
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


# 4: Basic function

In [4]:
def get_numpy_data( dataframe, features, output ):
    df = dataframe.copy()
    df['constant'] = 1 # this is how you add a constant column to a dataframe
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the dataframe df (now including constant):
    features_df = df[features]
    # the following line will convert the features_df into a numpy matrix
    input_matrix = features_df.as_matrix()
    # assign the column of df associated with the output to the output_df
    output_df = df[output]
    # the following will convert the output_df into a numpy array
    output_array = output_df.as_matrix()
    return (input_matrix, output_array)

In [5]:
def predict_output( input_matrix, weights ):
    predictions = np.dot( input_matrix, weights )
    return predictions

In [6]:
def print_coefficients( weights, features ):  
    labels = ['intercept'] + features
    df = pd.DataFrame({'name': labels, 'value': weights})
    return df

## <font color=red>Normalize features<font>
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [7]:
X = np.array( [[3.,5.,8.],[4.,12.,15.]] )
print X, type(X), X.shape, type(X.shape)

[[  3.   5.   8.]
 [  4.  12.  15.]] <type 'numpy.ndarray'> (2L, 3L) <type 'tuple'>


Numpy provides a shorthand for computing 2-norms of each column:

In [8]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [9]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [10]:
def normalize( feature_matrix ):
    norms = np.linalg.norm( feature_matrix, axis=0 )
    normalized_features = feature_matrix / norms
    return ( normalized_features, norms )

# 5: LASSO regression with coordinate descent

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## <font color=red>5.1. Play with rho<font>

Let us consider a simple model with 2 features:

In [11]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
( simple_feature_matrix, output ) = get_numpy_data( sales, simple_features, my_output )

Don't forget to normalize features:

In [12]:
simple_feature_matrix, norms = normalize( simple_feature_matrix )

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [13]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [14]:
prediction = predict_output( simple_feature_matrix, weights )

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [15]:
ro = np.zeros(3)
print ro, type(ro), range(len(ro))
prediction = predict_output( simple_feature_matrix, weights )
for i in range(len(ro)):
    feature_i = simple_feature_matrix[:,i]
    ro[i] =  sum( feature_i*(output - prediction + weights[i]*feature_i) )
print ro

[ 0.  0.  0.] <type 'numpy.ndarray'> [0, 1, 2]
[ 79414121.7965453   87962102.29102185  80982695.0210218 ]


## <font color=red>5.2. Single coordinate descent step<font>

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [16]:
def lasso_coordinate_descent_step( i, feature_matrix, output, weights, l1_penalty ):
    # compute prediction
    prediction = predict_output( feature_matrix, weights )
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature_i = feature_matrix[:,i]
    ro_i = sum( feature_i*(output - prediction + weights[i]*feature_i) )
    a = l1_penalty/2.
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -a:
        new_weight_i = ro_i + a
    elif ro_i > a:
        new_weight_i = ro_i - a
    else:
        new_weight_i = 0.
    
    return new_weight_i

## <font color=red>5.3. Coordinate descent<font>

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**

In [18]:
def lasso_regression_coordinate_descent( feature_matrix, output, initial_weights, l1_penalty, tolerance ):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    n = len(weights)
    while not converged:
        coordinate = np.zeros(n)
        for i in range(len(weights)):
            old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
            # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
            #     and old values for weight[i], ..., weight[d-1]
            weights[i] = lasso_coordinate_descent_step( i, feature_matrix, output, weights, l1_penalty )

            # use old_weights_i to compute change in coordinate
            coordinate[i] = np.abs( old_weights_i - weights[i] )
        if (coordinate < tolerance).sum() == n:
            converged = True
    return(weights)

## <font color=red>5.4. Single variable LASSO regression<font>

Using the following parameters, learn the weights on the sales dataset. 

In [19]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [20]:
( simple_feature_matrix, output ) = get_numpy_data( sales, simple_features, my_output )
( normalized_simple_feature_matrix, simple_norms ) = normalize( simple_feature_matrix )

Then, run your implementation of LASSO coordinate descent:

In [21]:
weights = lasso_regression_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
RSS = ((output - predict_output( normalized_simple_feature_matrix, weights ))**2).sum()

print 'The RSS for 1e7 penalty is', RSS
print_coefficients( weights, simple_features )

The RSS for 1e7 penalty is 1.63234909856e+15


,name,value
0,intercept,2.158283e+07
1,sqft_living,6.321846e+07
2,bedrooms,0.000000e+00


## <font color=red>5.5. Multiple variable LASSO regression<font>

Let us split the sales dataset into training and test sets.

In [22]:
(train_data, test_data) = train_test_split(sales, train_size=0.8, random_state=0)

Let us consider the following set of features.

In [23]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [25]:
( multiple_feature_matrix, output ) = get_numpy_data( train_data, all_features, my_output )
( normalized_multiple_feature_matrix, multiple_norms ) = normalize( multiple_feature_matrix )

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [26]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e7
tolerance = 1.0
weights1e7 = lasso_regression_coordinate_descent(normalized_multiple_feature_matrix, output,
                                                 initial_weights, l1_penalty, tolerance)
print_coefficients( weights1e7, all_features )

,name,value
0,intercept,2.459199e+07
1,bedrooms,0.000000e+00
2,bathrooms,0.000000e+00
3,sqft_living,4.833892e+07
4,sqft_lot,0.000000e+00
5,floors,0.000000e+00
6,waterfront,3.567998e+06
7,view,7.743713e+06
8,condition,0.000000e+00
9,grade,0.000000e+00


Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [27]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e8
tolerance = 1.0
weights1e8 = lasso_regression_coordinate_descent(normalized_multiple_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)
print_coefficients( weights1e8, all_features )

,name,value
0,intercept,7.138580e+07
1,bedrooms,0.000000e+00
2,bathrooms,0.000000e+00
3,sqft_living,0.000000e+00
4,sqft_lot,0.000000e+00
5,floors,0.000000e+00
6,waterfront,0.000000e+00
7,view,0.000000e+00
8,condition,0.000000e+00
9,grade,0.000000e+00


Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [28]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e4
tolerance = 5e5
weights1e4 = lasso_regression_coordinate_descent(normalized_multiple_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)
print_coefficients( weights1e4, all_features )

,name,value
0,intercept,7.768121e+07
1,bedrooms,-1.948653e+07
2,bathrooms,1.499397e+07
3,sqft_living,8.851634e+07
4,sqft_lot,-2.369359e+06
5,floors,-7.587057e+06
6,waterfront,6.724031e+06
7,view,7.498649e+06
8,condition,3.984470e+06
9,grade,1.379230e+07


## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [29]:
normalized_weights1e4 = weights1e4 / multiple_norms
normalized_weights1e7 = weights1e7 / multiple_norms
normalized_weights1e8 = weights1e8 / multiple_norms

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [30]:
( test_feature_matrix, test_output ) = get_numpy_data( test_data, all_features, 'price' )

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [31]:
RSS4 = ((test_output - predict_output( test_feature_matrix, normalized_weights1e4 ))**2).sum()
RSS7 = ((test_output - predict_output( test_feature_matrix, normalized_weights1e7 ))**2).sum()
RSS8 = ((test_output - predict_output( test_feature_matrix, normalized_weights1e8 ))**2).sum()
print 'The RSS for 1e4 penalty is', RSS4
print 'The RSS for 1e7 penalty is', RSS7
print 'The RSS for 1e8 penalty is', RSS8

The RSS for 1e4 penalty is 2.25586028306e+14
The RSS for 1e7 penalty is 2.72040828749e+14
The RSS for 1e8 penalty is 5.15598217189e+14
